In [1]:
import numpy as np
import open3d as o3d
import scipy.sparse as sp
from collections import defaultdict
from scipy import integrate


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#utils
import scipy.sparse as sp
def create_sprase_mat(dict:defaultdict[tuple[int, int], float], shape:tuple[int, int])->sp.coo_matrix:
    row_idxs:list[int] = []
    col_idxs:list[int] = []
    vals:list[float] = []

    for r_idx, c_idx in dict:
        row_idxs.append(r_idx)
        col_idxs.append(c_idx)
        vals.append(dict[(r_idx, c_idx)])
    
    return sp.coo_matrix((vals, (row_idxs, col_idxs)), shape=shape)


In [3]:
class String:
    def __init__(self, config:dict) -> None:
        self.config = config

    def __get_M_sh(self)->None:
        ret_data:defaultdict[tuple[int, int], float] = defaultdict(lambda : 0.0)
        Ns = self.config["Ns"]
        ls = self.config["ls"]
        rho_s = self.config["rho_s"]

        for i in range(Ns - 1):
            ret_data[(i, i)] += ls * rho_s / (Ns - 1)

        self.M_sh = create_sprase_mat(ret_data, (Ns-1, Ns-1))

    def __get_D_h(self)->None:
        ret_data:defaultdict[tuple[int, int], float] = defaultdict(lambda : 0.0)
        Ns = self.config["Ns"]
        T = self.config["T"]

        for i in range(Ns - 1):
            ret_data[(i, i)] += -T
            ret_data[(i, i+1)] += T

        self.D_h = create_sprase_mat(ret_data, (Ns-1, Ns))

    def __h_t(self, t)->float:
        assert t>=0.0, "t must be larger or equal to zero"
        t1 = self.config["t1"]
        t2 = self.config["t2"]
        if 0.0 <= t < t1:
            return 1.0 - np.cos(np.pi * t / t1)
        elif t < t2:
            return 1.0 + np.cos(np.pi * (t - t1) / (t2 - t1))
        else:
            return 0.0


    def get_f_sh(self, t)->None:
        x0 = self.config["x0"]
        delta_s = self.config["delta_s"]
        ls = self.config["ls"]
        Ns = self.config["Ns"]
        
        integrand = lambda x : np.exp(-(x-x0 / delta_s) ** 2)
        int_0_ls, error = integrand.quad(integrand, 0.0, ls)
        inv_int_0_ls = 1.0 / int_0_ls

        ht = self.__h_t(t)

        ret = np.zeros(Ns - 1, dtype=np.float64)

        for i in range(Ns - 1):
            ret[i] = integrand.quad(integrand, i * ls / (Ns - 1), (i + 1) * ls / (Ns - 1))

        ret *= ht * inv_int_0_ls
        self.f_sh = ret

    def __get_M_qh(self)->None:
        ls = self.config["ls"]
        Ns = self.config["Ns"]
        T  = self.config["T"]

        data:defaultdict[tuple[int, int], float] = defaultdict(lambda : 0.0)

        Q0 = 1 / (3 * T) * (ls / (Ns - 1)) ** 5
        Q1 = 1 / (6 * T) * (ls / (Ns - 1)) ** 5

        for i in range(Ns - 1):
            data[(i, i)] += Q0
            data[(i+1, i+1)] += Q0
            data[(i, i+1)] += Q1
            data[(i+1, i)] += Q1
        
        self.M_qh = create_sprase_mat(data, (Ns, Ns))